# Библиотеки

In [3]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd
from pandas_datareader import data as web # Reads stock data 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

import datetime as dt # For defining dates
import mplfinance as mpf # Matplotlib finance

import time

# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

# Объявляем дефолтные переменные 

In [4]:
# Define path to files
path = r"C:\Users\koala\Documents\CASE22\STOCKS/"

# Start date defaults
S_YEAR = 2017
S_MONTH = 10
S_DAY = 13
S_DATE_STR = f"{S_YEAR}-{S_MONTH}-{S_DAY}"
S_DATE_DATETIME = dt.datetime(S_YEAR, S_MONTH, S_DAY)

# End date defaults
E_YEAR = 2022
E_MONTH = 11
E_DAY = 10
E_DATE_STR = f"{E_YEAR}-{E_MONTH}-{E_DAY}"
E_DATE_DATETIME = dt.datetime(E_YEAR, E_MONTH, E_DAY)

# Создаем LIST из названий импортированных файлов

In [5]:
# listdir returns all files in the directory and isfile will return true
# if it is a file and then we store its name in our list named files
files = [x for x in listdir(path) if isfile(join(path, x))]

# Remove extension from file names
# Splitext splits the file name into 2 parts being the name and extension
# We say get all file names and then store just the name in our list named files
tickers = [os.path.splitext(x)[0] for x in files]
tickers

['AAIT',
 'AAL',
 'AAME',
 'AAOI',
 'AAON',
 'AAPL',
 'AAVL',
 'AAWW',
 'AAXJ',
 'ABAX',
 'ABCB',
 'ABCD',
 'ABCO',
 'ABIO',
 'ABMD',
 'ACAD',
 'ACET',
 'ACFC',
 'ACFN',
 'ACGL',
 'ACHC',
 'ACHN',
 'ACIW',
 'ACLS',
 'ACNB',
 'ACOR',
 'ACRX',
 'ACST',
 'ACTA',
 'ACTG',
 'ACUR',
 'ACWI',
 'ACWX',
 'ADBE',
 'ADES',
 'ADI',
 'ADMA',
 'ADMP',
 'ADP',
 'ADRA',
 'ADRD',
 'ADRE',
 'ADRU',
 'ADSK',
 'ADTN',
 'ADUS',
 'ADXS',
 'AEHR',
 'AEIS',
 'AERI',
 'AEY',
 'AEZS',
 'AFAM',
 'AFCB',
 'AFMD',
 'AFSI',
 'AGEN',
 'AGIO',
 'AGNC',
 'AGNCP',
 'AGND',
 'AGRX',
 'AGTC',
 'AGYS',
 'AGZD',
 'AHGP',
 'AHPI',
 'AIMC',
 'AIQ',
 'AIRR',
 'AIRT',
 'AKAM',
 'AKBA',
 'ALCO',
 'ALDX',
 'ALGN',
 'ALGT',
 'ALIM',
 'ALKS',
 'ALLT',
 'ALNY',
 'ALOG',
 'ALOT',
 'ALTR',
 'AMAT',
 'AMBA',
 'AMBC',
 'AMCF',
 'AMCX',
 'AMD',
 'AMED',
 'AMGN',
 'AMKR',
 'AMNB',
 'AMOT',
 'AMOV',
 'AMPH',
 'AMRK',
 'AMRN',
 'AMRS',
 'AMSC',
 'AMSF',
 'AMSWA',
 'AMTX',
 'AMWD',
 'AMZN',
 'ANAC',
 'ANCB',
 'ANCX',
 'ANDE',
 'ANGI',
 'ANG

# Dataframe из нашего List

In [7]:
stock_df = pd.DataFrame(tickers,columns=['Ticker'])
stock_df

,Ticker
0,AAIT
1,AAL
2,AAME
3,AAOI
4,AAON
...,...
1725,ZBRA
1726,ZEUS
1727,ZION
1728,ZIV


# Dataframe from CSV

In [8]:
# Reads a dataframe from the CSV file, changes index to date and returns it
def get_df_from_csv(ticker):
    
    # Try to get the file and if it doesn't exist issue a warning
    try:
        df = pd.read_csv(path + ticker + '.csv')
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df

# Saves Dataframe to CSV

In [9]:
def save_dataframe_to_csv(df, ticker):
    df.to_csv(path + ticker + '.csv')

# Удаляем Unnamed Columns в CSV 

In [10]:
def delete_unnamed_cols(df):
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df

# Return 1st Valid Date in Dataframe

In [75]:
def get_valid_dates(df, sdate, edate):  
    try:
        mask = (df['Date'] > sdate) & (df['Date'] <= edate) 
        sm_df = df.loc[mask]
        sm_df = sm_df.set_index(['Date'])
        sm_date = sm_df.index.min()
        last_date = sm_df.index.max()
        date_leading = '-'.join(('0' if len(x)<2 else '')+x for x in sm_date.split('-'))
        date_ending = '-'.join(('0' if len(x)<2 else '')+x for x in last_date.split('-'))
    except Exception:
        print("Date Corrupted")
    else:
        return date_leading, date_ending

#  Подсчет ROI за все время

In [76]:
def roi_between_dates(df, sdate, edate):
    try: 
        start_val = df.loc[sdate,'Adj Close'] 
        end_val = df.loc[edate,'Adj Close']
        roi = ((end_val - start_val) / start_val)
    except Exception:
        print("Data Corrupted")
    else:
        return roi

# Mean Between Dates

In [77]:
def get_mean_between_dates(df, sdate, edate):
    mask = (df['Date'] > sdate) & (df['Date'] <= edate)
    return df.loc[mask]["Adj Close"].mean()

# Стандартное отклонение между датами 

In [78]:
def get_sd_between_dates(df, sdate, edate):
    mask = (df['Date'] > sdate) & (df['Date'] <= edate)
    return df.loc[mask]["Adj Close"].std()

# Вариация между датами

In [79]:
def get_cov_between_dates(df, sdate, edate):
    mean = get_mean_between_dates(df, sdate, edate)
    sd = get_sd_between_dates(df, sdate, edate)
    return sd / mean

# Сложить активы в один датафрейм используя название столбца

In [121]:
def merge_df_by_column_name(col_name, sdate, edate, *tickers):
    # Will hold data for all dataframes with the same column name
    mult_df = pd.DataFrame()
    
    for x in tickers:
        df = get_df_from_csv(x)
        mask = (df.index >= sdate) & (df.index <= edate)
        mult_df[x] = df.loc[mask][col_name]
        
    return mult_df

# Выделяем сектора из каждой акции

In [69]:
# For Windows
sec_df = pd.read_csv(r'C:\Users\koala\Documents\CASE22\sectorrr.csv', on_bad_lines='skip')
sec_df



# Industrial
# Healthcare
# Information Technology 
# Communication
# Staples
# Discretionary
# Utilites
# Financials
# Materials
# Real Estate
# Energy
# ETF
# Services
# SPAC



,Symbol,Name,Sector
0,AAIT,iShares MSCI All Country Asia Information Tech...,ETF
1,AAL,American Airlines Group,Industrial
2,AAME,Atlantic American,Financials
3,AAOI,Applied Optoelectronics,Industrial
4,AAPL,Apple,Information Technology
...,...,...,...
1592,ZBRA,Zebra Technologies,Industrial
1593,ZEUS,Olympic Steel,Materials
1594,ZION,Zions Bancorporation NA,Financials
1595,ZIV,Credit Suisse AG,ETF


In [91]:
indus_df = sec_df.loc[sec_df['Sector'] == "Industrial"]
indus_df

,Symbol,Name,Sector
1,AAL,American Airlines Group,Industrial
3,AAOI,Applied Optoelectronics,Industrial
5,AAWW,Atlas Air Worldwide Holdings,Industrial
37,ADTN,ADTRAN,Industrial
43,AEY,ADDvantage Technologies Group,Industrial
...,...,...,...
1559,WIRE,Encore Wire,Industrial
1562,WLFC,Willis Lease Finance,Industrial
1565,WPRT,Westport Fuel Systems,Industrial
1575,WWD,Woodward,Industrial


# Returns a DF with ROIs for all Stocks

In [94]:
def get_rois_for_stocks(stock_df):
    # Will hold all tickers & stock rois
    tickers = []
    rois = []

    # iterrows provides the index and column for each row in the DF
    for index, row in stock_df.iterrows():
        df = get_df_from_csv(row['Symbol'])
    
        # If we can't find the ticker delete it from the dataframe
        if df is None:
            pass
            # print(row['Symbol'], " is not available")
        else:
            tickers.append(row['Symbol'])
            sdate, edate = get_valid_dates(df, '2017-13-10', '2022-11-10')
            df = df.set_index(['Date'])
            roi = roi_between_dates(df, sdate, edate)
            rois.append(roi)
    return pd.DataFrame({'Ticker':tickers, 'ROI':rois})

# Считаем ROI для каждого актива и разбиваем их по сектору

In [96]:
industrial = get_rois_for_stocks(indus_df)
health_care = get_rois_for_stocks(health_df)
it = get_rois_for_stocks(it_df)
commun = get_rois_for_stocks(comm_df)
staple = get_rois_for_stocks(staple_df)
discretion = get_rois_for_stocks(discretion_df)
utility = get_rois_for_stocks(utility_df)
finance = get_rois_for_stocks(financial_df)
material = get_rois_for_stocks(material_df)
restate = get_rois_for_stocks(restate_df)
energy = get_rois_for_stocks(energy_df)

etf = get_rois_for_stocks(etf_df)
services = get_rois_for_stocks(services_df)
spac = get_rois_for_stocks(spac_df)


# TOP-3 industrial

In [98]:
industrial.sort_values(by=['ROI'], ascending=False).head(3)

,Ticker,ROI
40,CLFD,6.320332
171,PLUG,6.028925
149,NSSC,5.460116


# TOP-3 Healthcare

In [99]:
health_care.sort_values(by=['ROI'], ascending=False).head(3)

,Ticker,ROI
250,TNDM,14.224409
265,XENE,11.104348
36,ARWR,8.336021


# TOP-3 IT

In [101]:
it.sort_values(by=['ROI'], ascending=False).head(3)

,Ticker,ROI
62,ENPH,117.942312
103,LSCC,9.549153
7,AEHR,7.378571


# TOP-3 Communications

In [102]:
commun.sort_values(by=['ROI'], ascending=False).head(3)

,Ticker,ROI
35,PERI,6.970874
20,IRDM,3.566379
46,TTGT,2.333333


# TOP-3 Staples

In [104]:
staple.sort_values(by=['ROI'], ascending=False).head(3)

,Ticker,ROI
10,FRPT,2.350500
7,COST,1.894745
29,SMPL,1.651515


# TOP-3 Discretionary

In [105]:
discretion.sort_values(by=['ROI'], ascending=False).head(3)

,Ticker,ROI
113,TSLA,7.925217
21,CROX,6.121048
50,HEAR,3.713483


# TOP-3 Utilities

In [106]:
utility.sort_values(by=['ROI'], ascending=False).head(3)

,Ticker,ROI
4,MSEX,1.486159
2,CWCO,0.590256
0,ARTNA,0.573002


# TOP-3 Finance

In [107]:
finance.sort_values(by=['ROI'], ascending=False).head(3)

,Ticker,ROI
12,ATLC,10.057143
6,AMRK,4.313258
156,LPLA,3.472111


# TOP-3 Materials

In [108]:
material.sort_values(by=['ROI'], ascending=False).head(3)

,Ticker,ROI
22,TGLS,2.766031
10,HWKN,1.616835
21,STLD,1.417639


# TOP-3 Real Estate

In [109]:
restate.sort_values(by=['ROI'], ascending=False).head(3)

,Ticker,ROI
9,INTG,1.130435
11,LAND,0.884137
16,SBAC,0.873506


# TOP-3 Energy

In [110]:
energy.sort_values(by=['ROI'], ascending=False).head(3)

,Ticker,ROI
0,AMTX,10.448276
34,VTNR,8.728260
29,REGI,4.168067


# TOP-3 etf

In [111]:
etf.sort_values(by=['ROI'], ascending=False).head(3)

,Ticker,ROI
143,LIME,3.335714
211,TAPR,3.234329
69,DTYS,2.791624


# TOP-3 services

In [112]:
services.sort_values(by=['ROI'], ascending=False).head(3)

,Ticker,ROI
5,EXPO,2.009593
3,CSGP,1.835675
2,CRAI,1.806465


# TOP-3 spac

In [113]:
spac.sort_values(by=['ROI'], ascending=False).head(3)

,Ticker,ROI
3,BRLI,0.090256
7,ROSE,0.035703
0,ADRA,0.028718


# Наш портфель

In [117]:
port_list = port_list = ['CLFD',  'PLUG',  'NSSC',  'TNDM',  'XENE',  'ARWR',  'ENPH',  'LSCC',  'AEHR',  'PERI',  'IRDM',  'TTGT',  'FRPT',  'COST',  'SMPL',  'TSLA',  'CROX',  'HEAR',  'MSEX',  'CWCO',  'ARTNA',  'ATLC',  'AMRK',  'LPLA',  'TGLS',  'HWKN',  'STLD',  'INTG',  'LAND',  'SBAC',  'AMTX',  'VTNR',  'REGI',  'LIME',  'TAPR',  'DTYS',  'EXPO',  'CSGP',  'CRAI',  'BRLI',  'ROSE',  'ADRA']
num_stocks = len(port_list)
num_stocks

42

# Подбиваем все в цены в один файл